In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torch.nn import functional as F
from torchprofile import profile_macs  # Assumes torchprofile is installed
from torchvision.models import ResNet18_Weights

# Step 1: Define a new model with softmax appended in the forward pass
class ResNet18WithSoftmax(nn.Module):
    def __init__(self):
        super(ResNet18WithSoftmax, self).__init__()
        # Use weights instead of pretrained for loading model weights
        self.model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

    def forward(self, x):
        x = self.model(x)
        return F.softmax(x, dim=1)  # Apply softmax on the output layer

# Instantiate the modified model
model_with_softmax = ResNet18WithSoftmax()

# Step 2: Calculate FLOPs
# Example input tensor of the shape that ResNet18 expects (batch size = 1, 3 color channels, 224x224 image)
input_tensor = torch.randn(1, 3, 224, 224)

# Profile the model to calculate MACs/FLOPs
flops = profile_macs(model_with_softmax, input_tensor) * 2  # Convert MACs to FLOPs

# Convert FLOPs to GFLOPs
gflops = flops / 1e9
print(f"Total GFLOPs for inference: {gflops:.2f} GFLOPs")
